In [2]:
import torch 
from torch import nn, optim 
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader

# Use of GPU otherwise CPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# List of transformation
tf = [
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
]

# Opens en applies transformations established previously
train_dl = DataLoader(
    datasets.ImageFolder('data/Training', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True 
)

test_dl = DataLoader(
    datasets.ImageFolder('data/Testing', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True 
)